In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [100]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
# ...these are taken from predictions for the Run-5 CMS
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")
run5_smartpx_system = dataframes_from_spreadsheet("cms_system_200_smartpx.xlsx")

In [3]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [4]:
run5_system.triggers

,Name,Output,Data (bytes),Reduction Ratio,Classifier,Skill mean,Skill variance,Link Efficiency (J/bit),Op Efficiency (J/op),Compression
0,Tracking,Intermediate,0,1.0,Dummy,0,0,2.500000e-11,0.000,0
1,Timing,Intermediate,0,1.0,Dummy,0,0,2.500000e-11,0.000,0
2,Calorimetry,Intermediate,0,1.0,Dummy,0,0,2.500000e-11,0.000,0
3,Muon,Intermediate,0,1.0,Gaussian,0,0,2.500000e-11,0.000,0
4,Intermediate,Global,260000,53.3,L1T,0,0,2.500000e-11,0.003,0
5,Global,Disk,0,100.0,Gaussian,4,1,2.500000e-11,16.000,0
6,Disk,NaN,0,1.0,Dummy,0,0,2.500000e-11,0.000,0


In [5]:
#calculate the total reduction to keep this constant across experiments
overall_reduction = run5_system.triggers.iloc[4]["Reduction Ratio"] * run5_system.triggers.iloc[5]["Reduction Ratio"]

In [6]:
overall_reduction

5330.0

In [7]:
40e6 / overall_reduction

7504.6904315197

In [8]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [9]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [10]:
"""
Vary the accept rate of the level 1 trigger and inspect its impact on performance and resources required
"""
def init_system(system: System, functions, reduction: float, skill: float):
    t = system.triggers.copy()
    #intermediate reduction stage
    t.at[4, "Reduction Ratio"] = reduction
    #global reduction stage
    t.at[5, "Reduction Ratio"] = overall_reduction / reduction
    #change L1T skill
    t.at[4, "Skill mean"] = skill
    g = construct_graph(system.detectors, t, system.globals, functions)

    return g

In [11]:
reduction_to_ratio(0.0)

1.0

In [115]:
ex_gpu = init_system(run5_system, funcs_gpu, 53.3, 3.0)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



In [113]:
ex_reduction = init_system(run5_smartpx_system, funcs_gpu, 53.3, 3.0)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



In [12]:
#current l1t accept / skill
ex = init_system(run5_system, funcs, 400, 3)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [13]:
ex.nodes["Intermediate"]["reduction ratio"]

400.0

In [14]:
ex.nodes["Intermediate"]["classifier"].skill_boost

0.0

In [15]:
ratio_to_reduction(ex.nodes["Intermediate"]["reduction ratio"])

0.9975

In [16]:
ex.graph["op power"] / 1e6

44.967879752467255

In [17]:
ex.nodes["Intermediate"]["output rate"]

99682

In [18]:
ex.nodes["Intermediate"]["input rate"] / ex.nodes["Intermediate"]["output rate"]

401.2760478321061

In [19]:
ex.nodes["Intermediate"]["input rate"]

39999999

In [20]:
ex.nodes["Global"]["input rate"] / ex.nodes["Global"]["output rate"]

13.328252440165798

In [21]:
ex.nodes["Global"]["output rate"]

7479

In [22]:
ex.nodes["Global"]["input rate"]

99682

In [23]:
from copy import deepcopy

In [103]:
def extract_results(graph):

    power = graph.graph["op power"] + graph.graph["link power"]
    confusion = graph.graph["performance"]

    return power, confusion

In [24]:
def vary_system(graph, reduction_ratio: float, skill: float):
    graph = deepcopy(graph)
    graph.nodes["Intermediate"]["reduction ratio"] = reduction_ratio
    graph.nodes["Intermediate"]["classifier"].skill_boost = skill

    graph = update_throughput(graph)

    power = graph.graph["op power"] + graph.graph["link power"]
    confusion = graph.graph["performance"]

    return power, confusion, graph

In [123]:
baseline = vary_system(ex, 400, 0.0)

In [112]:
tracking_l1t = vary_system(ex, 400, 0.4)

In [116]:
gpu_system = vary_system(ex_gpu, 53.3, 0.0)
phase2_system = vary_system(ex_gpu, 53.3, 0.4)

In [117]:
smart_system = vary_system(ex_reduction, 53.3, 0.4)

In [131]:
system_labels = ["Phase-1", "L1T Tracking", "GPU HLT", "L1T Tracking + GPU HLT", "L1T Tracking, GPU HLT, & Data Reduction"]

In [125]:
all_systems = [baseline, tracking_l1t, gpu_system, phase2_system, smart_system]

In [25]:
r = vary_system(ex, 100, 0.0)

In [26]:
r[0] / 1e6

179.22039064864614

In [27]:
#its predicted confusion matrix for the workflow
r[1]

array([[39951695,    18253],
       [   18284,    11763]])

In [42]:
#vary this accept rate from today's rate to the planned Run-5 
l1t_reductions = np.linspace(450, 40, 101)
l1t_skills = np.linspace(0, 1.0, 101)

In [43]:
res = [[vary_system(ex, r, s) for r in l1t_reductions] for s in l1t_skills]

In [79]:
def recall(matrix):
    tp = matrix[1,1]
    fn = matrix[0,1]
    return tp / (tp + fn)

In [80]:
def extract_metrics(results):
    all_confusion = np.array([r[1] for r in results])

    all_power = [r[0] / density_scale_model(r[2].graph["globals"]["Year"][0]) for r in results]
    all_power = np.array(all_power)

    all_recall = np.array([recall(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    all_precision = np.array([precision(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    all_f1 = np.array([f1_score(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    productivity = np.array([np.sum(get_passed(all_confusion[i,:,:])) for i in range(all_confusion.shape[0])])

    metrics = {"confusion": all_confusion,
               "power": all_power,
               "recall": all_recall,
               "precision": all_precision,
               "f1 score": all_f1,
               "productivity": all_recall * productivity}

    return metrics

In [81]:
run5_metrics = [extract_metrics(r) for r in res]

In [82]:
res_f1 = np.stack([r["f1 score"] for r in run5_metrics])

In [83]:
res_recall = np.stack([r["recall"] for r in run5_metrics])

In [84]:
res_precision = np.stack([r["precision"] for r in run5_metrics])

In [85]:
power = np.stack([r["power"] for r in run5_metrics])

In [86]:
from scipy.ndimage import gaussian_filter

In [91]:
smoothed_f1 = gaussian_filter(res_f1, sigma=3)

In [92]:
np.savez_compressed("smoothed_f1.npz", smoothed_f1)

In [126]:
systems_f1 = np.array([f1_score(s[1]) for s in all_systems])

In [157]:
systems_power = np.array([s[0] / density_scale_model(s[2].graph["globals"]["Year"][0]) for s in all_systems])

In [159]:
systems_power / 1e6

array([ 6.93759447,  6.94377088, 25.85367308, 25.84650157, 20.48202025])

In [127]:
systems_reductions = np.array([400, 400, 53.3, 53.3, 53.3])
l1t_improvement = np.array([0.0, 0.4, 0.0, 0.4, 0.4])

In [151]:
fig = go.Figure(data =
    go.Contour(
        z=smoothed_f1,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
        contours = dict(showlabels = True),
        colorbar = dict(title = "F1 Score")
         
    ),
    )

fig.add_trace(go.Scatter(x = systems_reductions[0:1],
                        y = l1t_improvement[0:1],
                        mode = "markers",
                        marker = dict(size = 14, color = "gray"),
                        name = "Phase-1"))

fig.add_trace(go.Scatter(x = systems_reductions[1:2],
                        y = l1t_improvement[1:2],
                        mode = "markers",
                        marker = dict(size = 14, color = "red"),
                        name = "L1T Tracking"))

fig.add_trace(go.Scatter(x = systems_reductions[2:3],
                        y = l1t_improvement[2:3],
                        mode = "markers",
                        marker = dict(size = 14, color = "blue"),
                        name = "Increased L1T Accept"))

fig.add_trace(go.Scatter(x = systems_reductions[3:4],
                        y = l1t_improvement[3:4],
                        mode = "markers",
                        marker = dict(size = 14, color = "purple"),
                        name = "Phase-2 & Data Reduction"))

# fig.add_trace(go.Scatter(x = systems_reductions[0:4],
#     y = l1t_improvement[0:4],
#     mode = "markers+text",
#     marker = dict(size = 14,
#             color = ["white", "blue", "red", "purple"]),
#     text =  system_labels[0:4],
#     textposition = "top left",
#     textfont = dict(color = "rgb(255, 255, 255)")       ))

fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "L1T Skill Improvement",
                  title = "F1 Score by L1T Skill & Reduction Ratio",
                  legend=dict(xanchor = "right",
                    x = 0.95))
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(range=[0.0, 0.8])
fig.show()

In [52]:
fig = go.Figure(data =
    go.Contour(
        z=power,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800, 
                  height = 600,
                  title = "Power by Trigger Skill & Reduction Ratio",
                  xaxis=dict(
                        title="Reduction Ratio",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the x-axis title
                    ),
                    yaxis=dict(
                        title="Skill",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the y-axis title
                    ),
                    font = dict(size=18,),)
fig.update_xaxes(autorange="reversed")
fig.show()

In [53]:
#because its rejection is so much higher, there's more potential improvement gained by making L1T's skill higher 
#than simply passing more data to the HLT

In [63]:
fig = go.Figure(data =
    go.Contour(
        z = smoothed_f1,
        x=power[0,:], # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
         contours = dict(showlabels = True)
    ),
    )
fig.update_layout(width = 800,
                  height = 600,
                  title = "F1 Score by Skill Improvement and Power",
                  xaxis_title = "Power",
                  yaxis_title = "L1T Skill Improvement")
fig.update_yaxes(range=(0.0, 0.8))
fig.show()

In [161]:
fig = go.Figure(data =
    go.Contour(
        z = np.transpose(smoothed_f1),
        y=power[0,:], # horizontal axis
        x=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
         contours = dict(showlabels = True)
    ),
    )

fig.add_trace(go.Scatter(y = systems_power[0:1],
                        x = l1t_improvement[0:1],
                        mode = "markers",
                        marker = dict(size = 14, color = "gray"),
                        name = "Phase-1"))

fig.add_trace(go.Scatter(y = systems_power[1:2],
                        x = l1t_improvement[1:2],
                        mode = "markers",
                        marker = dict(size = 14, color = "red"),
                        name = "L1T Tracking"))

fig.add_trace(go.Scatter(y = systems_power[2:3],
                        x = l1t_improvement[2:3],
                        mode = "markers",
                        marker = dict(size = 14, color = "blue"),
                        name = "Increased L1T Accept"))

fig.add_trace(go.Scatter(y = systems_power[3:4],
                        x = l1t_improvement[3:4],
                        mode = "markers",
                        marker = dict(size = 14, color = "purple"),
                        name = "Phase-2"))

fig.add_trace(go.Scatter(y = systems_power[4:],
                        x = l1t_improvement[4:],
                        mode = "markers",
                        marker = dict(size = 14, color = "green"),
                        name = "Data Reduction"))

fig.update_layout(width = 800,
                  height = 600,
                  title = "F1 Score by Skill Improvement and Power",
                  yaxis_title = "Power",
                  xaxis_title = "L1T Skill Improvement",
                  legend=dict(xanchor = "right",
                    x = 0.95))
fig.update_xaxes(range=(0.0, 0.7))
fig.show()

In [171]:
output_rate = np.array([1e3, 1e3, 7.5e3, 7.5e3, 7.5e3])

In [172]:
output_rate

array([1000., 1000., 7500., 7500., 7500.])

In [173]:
productivity = (systems_f1 * output_rate) / systems_power

In [174]:
systems_f1

array([0.27808558, 0.49943292, 0.44246019, 0.83918694, 0.83974957])

In [175]:
productivity

array([4.00838614e-05, 7.19253164e-05, 1.28355123e-04, 2.43510791e-04,
       3.07495143e-04])

In [176]:
fig = go.Figure(data =
    go.Bar(
        x = ["Phase-1", "L1T Tracking", "Increased L1T Accept", "Phase-2", "Data Reduction"],
        y= productivity
    ),
    )



fig.update_layout(width = 800,
                  height = 600,
                  title = "Productivity by System",
                  yaxis_title = "Productivity (Relevant Samples per Joule)",
                  xaxis_title = "System", )

fig.show()